In [183]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import os

file_path = '/home/gaston/Escritorio/env_laburo/src/scrapingTrabajo/scrap_canasta_basica.py/ASADO.HTM'


In [184]:
# Leer el archivo HTM
with open(file_path, 'r', encoding='latin-1') as file:
    html_content = file.read()

# Crear el objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')


# Encontrar todos los elementos <table>
tables = list(soup.find_all('table'))


<h3> Busqueda de datos utiles </h3>

In [185]:
#Configuracion del navegador
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Ruta completa al archivo HTML

# Convertir la ruta a una URL válida para Selenium con el prefijo 'file://'
file_url = f'file://{os.path.abspath(file_path)}'

print(file_url)

# Cargar el archivo HTML local
driver.get(file_url)

#Buscamos por anio el mes, anio, y otros datos utiles | Encontrados se elimina el espacio en blanco sobrante
mes = driver.find_element(By.XPATH, '/html/body/table[1]/tbody/tr[12]/td[14]/font').text.strip()
anio = driver.find_element(By.XPATH, '/html/body/table[1]/tbody/tr[12]/td[10]/font').text.strip()
codigo_variedad = driver.find_element(By.XPATH,'/html/body/table[1]/tbody/tr[12]/td[18]/font').text.strip()
tipo_producto = driver.find_element(By.XPATH,'/html/body/table[1]/tbody/tr[12]/td[20]/font').text.strip()


# Cerrar el navegador
driver.quit()

file:///home/gaston/Escritorio/env_laburo/src/scrapingTrabajo/scrap_canasta_basica.py/ACEITE1.HTM


Formateando tablas

In [186]:
lista_fonts = tables[0].find_all('font')

lista_fonts = lista_fonts[46:-4]

lista_fonts_limpia = [font.get_text(strip=True) for font in lista_fonts]


In [187]:
# Iterar sobre el conjunto en bloques de 18 elementos

#Lista de listas, cada lista representa una fila de datos
lista_acum_datos = []

#Indice de busqueda
i = 0

#Recorremos la lista, buscando cada 18 elementos
while i < len(lista_fonts_limpia):

    lista_aux = lista_fonts_limpia[i:i+18]
    lista_acum_datos.append(lista_aux)

    #Si le sumo 20 a i, entonces es como hacer ese salto de dos lineas
    i +=20

In [188]:
lista_fonts_segunda_tabla = tables[1].find_all('font')
lista_fonts_segunda_tabla[42:-4]

lista_fonts_limpia_segunda_tabla = [font.get_text(strip=True) for font in lista_fonts_segunda_tabla]

In [189]:
#Indice de busqueda
i = 0

#Recorremos la lista, buscando cada 18 elementos
while i < len(lista_fonts_limpia_segunda_tabla):

    lista_aux = lista_fonts_limpia[i:i+18]
    lista_acum_datos.append(lista_aux)

    #Si le sumo 20 a i, entonces es como hacer ese salto de dos lineas
    i +=20


In [190]:
for i in lista_acum_datos:
    print(i)

['1250003', 'H', '35', '4', '1', '1', '1', '2', '2469.00', '', '2222.10', '2599.00', '', '2599.00', '-14.5', 'B', 'P', '10']
['1250003', 'H', '35', '14', '2', '1', '1', '2', '3055.00', '', '2749.50', '2469.00', '', '2222.10', '23.7', 'B', 'B', '136- Marca: NATURA \\ ---| 45- CC: 1500 \\ ---| 245- Variante: GIRASOL \\ ---']
['1250004', 'H', '35', '6', '1', '1', '1', '2', '1849.00', '', '3081.67', '1700.00', '', '2833.33', '8.8', 'P', 'P', '136- Marca: LIRA \\ ---| 45- CC: 900 \\ ---| 245- Variante: GIRASOL \\ ---']
['1250004', 'H', '35', '16', '2', '1', '1', '2', '1900.00', '', '3166.67', '1849.00', '', '3081.67', '2.8', 'P', 'P', '']
['1250007', 'S', '35', '9', '1', '1', '1', '2', '1599.00', '', '2665.00', '1699.00', '', '2831.67', '-5.9', 'O', 'P', '']
['1250007', 'S', '35', '19', '2', '1', '1', '2', '1699.00', '', '2831.67', '1599.00', '', '2665.00', '6.3', 'P', 'O', '136- Marca: COCINERO \\ ---| 45- CC: 900 \\ ---| 245- Variante: CLASICO \\ ---']
['1250008', 'S', '35', '5', '1', '1'

In [191]:
df = pd.DataFrame(data=lista_acum_datos)

df = df.drop([9,12],axis=1)

lista_columnas = ['identificador','tipo_local','']


df['fecha'] = pd.to_datetime(anio+"-"+mes+"-"+"01")
df['codigo_variedad'] = codigo_variedad
df['tipo_producto'] = tipo_producto

df

,0,1,2,3,4,5,6,7,8,10,11,13,14,15,16,17,fecha,codigo_variedad,tipo_producto
0,1250003,H,35,4,1,1,1,2,2469.00,2222.10,2599.00,2599.00,-14.5,B,P,10,2024-08-01,1131101,Aceite de girasol
1,1250003,H,35,14,2,1,1,2,3055.00,2749.50,2469.00,2222.10,23.7,B,B,136- Marca: NATURA \ ---| 45- CC: 1500 \ ---| ...,2024-08-01,1131101,Aceite de girasol
2,1250004,H,35,6,1,1,1,2,1849.00,3081.67,1700.00,2833.33,8.8,P,P,136- Marca: LIRA \ ---| 45- CC: 900 \ ---| 245...,2024-08-01,1131101,Aceite de girasol
3,1250004,H,35,16,2,1,1,2,1900.00,3166.67,1849.00,3081.67,2.8,P,P,,2024-08-01,1131101,Aceite de girasol
4,1250007,S,35,9,1,1,1,2,1599.00,2665.00,1699.00,2831.67,-5.9,O,P,,2024-08-01,1131101,Aceite de girasol
5,1250007,S,35,19,2,1,1,2,1699.00,2831.67,1599.00,2665.00,6.3,P,O,136- Marca: COCINERO \ ---| 45- CC: 900 \ ---|...,2024-08-01,1131101,Aceite de girasol
6,1250008,S,35,5,1,1,1,2,2939.00,2939.00,2939.00,2939.00,0.0,P,P,,2024-08-01,1131101,Aceite de girasol
7,1250008,S,35,15,2,1,1,2,2939.00,2939.00,2939.00,2939.00,0.0,P,P,,2024-08-01,1131101,Aceite de girasol
8,1250009,S,35,2,1,1,1,2,1699.00,2831.67,1784.00,2973.33,-4.8,P,P,136- Marca: COCINERO \ ---| 45- CC: 900 \ ---|...,2024-08-01,1131101,Aceite de girasol
9,1250009,S,35,12,2,1,1,2,1699.00,2831.67,1699.00,2831.67,0.0,P,P,,2024-08-01,1131101,Aceite de girasol
